In [95]:
import os
import pandas as pd
import numpy as np

from nptdms import TdmsFile
from IPython.display import display

In [96]:
# Função para ler os arquivos tdms e montar o dataframe do pandas
def read_tdms_file(filename):
    tdms_file = TdmsFile.read(filename)
    
    if len(tdms_file.groups()) > 1:
        print("PROBLEMA")
    
    dataframes = []
    for group in tdms_file.groups():
        for channel in group.channels():
            dataframes.append(channel.as_dataframe())
            
    
    df = pd.concat(dataframes, axis=1)
    df.columns = [column.split('/')[-1][1:-1] for column in df.columns]
    return df



In [97]:
directory = '../data/argonne/'
files = os.listdir(directory)
files.sort()
trips_ice = []
for file in files:
    trip = read_tdms_file(directory + file)
    trip = trip.sort_values(by='Time')
    trip = trip.rename(columns={'Test_ID': 'Trip'})
    trips_ice.append(trip)

trips_ice = pd.concat(trips_ice)
trips_ice['Dyno_Spd'] = trips_ice['Dyno_Spd'] * 1.60934 # convert from miles into km
trips_ice.shape

(183749, 216)

In [98]:
maf = trips_ice['Eng_MAF_total_ECM']
direct = trips_ice['Eng_FuelFlow_Direct']

fuel_flow = np.where(maf>0, ((maf/(14.7*740))), direct) # (Litro/h)
K = trips_ice['Dyno_Spd']/fuel_flow 

fe = 1510
fg = 2310
P = 28

e_ice = (1/K)*((((100-P)/100)*fg) + ((P/100)*fe))
trips_ice['CO2 Emissions'] = (e_ice * trips_ice['Dyno_Spd']) / 3600



In [99]:
colunas_selecionadas = [
    'Time', 
    'Dyno_Spd',
    'Eng_throttle_pos_ECM',
    'Eng_torque_TCM',
    'CO2 Emissions',
    'Cell_Temp',
    'HVAC_pass_compartment_air_temp_filt_HVAC',
    'Radiator_Air_Outlet_Temp',
    'ActVehAccel',
    'Trip'
]

# Terminando o préprocessamento de problemas no dataset
trips_ice = trips_ice[colunas_selecionadas]
trips_ice = trips_ice[trips_ice['Trip'] != 61177923.13997565]
trips_ice['Trip'] = trips_ice['Trip'].astype(int).astype(str)
trips_ice = trips_ice[trips_ice['Time'] >= 0].reset_index(drop=True)
trips_ice = trips_ice.rename(columns={
    'Cell_Temp': 'Ambient Temperature [°C]',
    'HVAC_pass_compartment_air_temp_filt_HVAC': 'Cabin Temperature Sensor [°C]',
    'Radiator_Air_Outlet_Temp': 'Heat Exchanger Temperature [°C]',
    'ActVehAccel': 'Longitudinal Acceleration [m/s^2]',
    'Eng_throttle_pos_ECM': 'Throttle [%]',
    'Dyno_Spd': 'Velocity [km/h]', 
    'Eng_torque_TCM': 'Motor Torque [Nm]', 
    'Time': 'Time [s]'
})
trips_ice = trips_ice.dropna()
trips_ice

,Time [s],Velocity [km/h],Throttle [%],Motor Torque [Nm],CO2 Emissions,Ambient Temperature [°C],Cabin Temperature Sensor [°C],Heat Exchanger Temperature [°C],Longitudinal Acceleration [m/s^2],Trip
1,0.199981,0.000045,8.462286,31.560106,0.000282,21.650933,28.000358,62.130068,0.0,61908011
2,0.299981,0.000010,8.982816,43.973133,0.000267,21.651104,28.000358,62.123342,0.0,61908011
8,0.899980,0.000045,8.500086,32.710014,0.000284,21.650968,28.000358,62.137541,0.0,61908011
9,0.999980,0.000055,8.830984,43.317326,0.000261,21.651258,28.000358,62.137925,0.0,61908011
10,1.099980,0.000010,8.938069,38.450241,0.000278,21.650807,28.000358,62.139717,0.0,61908011
...,...,...,...,...,...,...,...,...,...,...
181529,1228.298810,0.000055,29.466669,-19.250000,0.000010,22.469514,0.000000,60.525775,0.0,61908024
181530,1228.398809,0.000012,29.458800,-19.250000,0.000010,22.469514,0.000000,60.515894,0.0,61908024
181533,1228.698809,0.000043,29.498000,-19.250000,0.000010,22.477966,0.000000,60.562923,0.0,61908024
181534,1228.798809,0.000055,29.467005,-19.250000,0.000010,22.479637,0.000000,60.583212,0.0,61908024


In [100]:
trips_ice.to_csv('../data/combustao_argonne.csv', index=False)

In [126]:
len(trips_ice['Trip'].unique())

10